In [1]:
from __future__ import division
from IPython.display import display
from IPython.display import Markdown as md
from IPython.core.display import SVG
from sympy.interactive import printing
printing.init_printing(use_latex='mathjax')

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.odr import *
import scipy.odr.odrpack as odrpack
from uncertainties import ufloat
from uncertainties.umath import *
from uncertainties import unumpy

import sympy as sp
sf = sp.sympify

import scipy.constants as cm
import pint as pn
ureg = pn.UnitRegistry(system='mks')
ureg.default_format = '~P'
pe = ureg.parse_expression

## Konstanten und Einheiten
CM = pd.DataFrame(cm.physical_constants).T
CM['c'] = CM.index
CM = CM.reset_index(drop=True)
def getpc(quant):
    DF = CM
    DF = DF[DF['c']==quant].reset_index(drop=True)
    quan = DF[0][0] * ureg(DF[1][0])
    return quan
def addnoise(X,scale=0.01,lin=False):
    if not lin:
        mag = scale*(np.max(X.m)-np.min(X.m))
        return np.random.normal(0,mag,len(X.m))*X.u
    if lin:
        return np.random.normal(0,1,len(X.m))*X*scale
def uODR(func,beta0,xdata,ydata,**kwargs):
    xfit = kwargs.get('xfit',xdata)
    sx = kwargs.get('sx',None)
    sy = kwargs.get('sy',None)
    model = Model(func)
    data = RealData(xdata,ydata, sx=sx, sy=sy)
    odr = ODR(data, model, beta0=beta0)
    out = odr.run()
    # Estimated parameter values, of shape q
    p = out.beta
    # Standard errors of the estimated parameters, of shape p
    perr = out.sd_beta
    up =unumpy.uarray(p, perr)
    yfit=func(p, xfit) 
    return up , p, yfit, out

In [2]:
g = getpc('standard acceleration of gravity')
m_e = getpc('electron mass')
e = getpc('elementary charge')
c = getpc('speed of light in vacuum')
e0 = getpc('electric constant')
mu0 = getpc('mag. constant')

In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Quellcode an/aus schalten."></form>''')

# Übungen: Elektromagnetische Wellen vom Dipol


### Nachfeld und Fernfeld
<img src="https://www.schulphysik.ch/inline/html/ElektromagnetischeWellen/Bilder/Hertzdipol%201.gif">
<img src="https://www.schulphysik.ch/inline/html/ElektromagnetischeWellen/Bilder/Hertzdipol%202.gif">
Quelle:
https://www.schulphysik.ch/inline/html/ElektromagnetischeWellen/

### 2D-Abstrahlung

In [4]:
from IPython.display import IFrame

IFrame(src='https://www.didaktikonline.physik.uni-muenchen.de/programme/dipolstr/Dipolstr_leifi.html', width=500, height=500)

### Elektromagnetische Welle im Fernfeld
<img src="https://upload.wikimedia.org/wikipedia/commons/9/99/EM-Wave.gif">

# Stehende elektromagnetische Wellen
Trifft eine elektromagnetische Welle auf eine Metallplatte, dann entsteht an der Metalloberfläche ein Knoten des E-Feldes. In dem Metall wird das "externe" elektrische Feld durch ein Gegenfeld kompensiert und die Feldstärke im Metall ist 0. Es entsteht eine Reflexion der Welle (Phasensprung um $\pi$). Die einlaufende und die reflektierte Welle überlagern sich konstruktiv - es entsteht hinter der Wand eine stehende Welle. Im Abstand $\tfrac{\lambda}{4}$ von der Wand entfernt kann man einen Bauch in dem E-Feld messen. Danach misst man alle $\tfrac{\lambda}{4} + n \cdot \tfrac{\lambda}{2}$ einen Bauch.

In [5]:
from IPython.display import IFrame

IFrame(src='./material/08_E_.html', width=700, height=250)

Das Magnetfeld der elektromagnetischen Welle wird jedoch ohne Phasensprung reflektiert. Daraus ergibt sich, dass man direkt an der Metallplatte ein Bauch des B-Feldes misst und dann alle $\tfrac{\lambda}{2}$ von der Metallplatte entfernt.

In [6]:
from IPython.display import IFrame

IFrame(src='./material/08_B_.html', width=700, height=250)

Das E-Feld und das B-Feld der stehenden Welle sind also zeitlich in Phase (die eine Welle ist nicht schneller als die Andere) - jedoch sind sie nicht räumlich in Phase, sondern sind zueinander um $\tfrac{\lambda}{4}$ versetzt.
<img src="./material/08_standing-EM.png">
Quelle:
https://web.mit.edu/viz/EM/visualizations/coursenotes/modules/guide13.pdf